# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()


In [2]:
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'lxml')
table = soup.find('table', class_='TABLE_G')
print(table)

<table align="Center" border="1" cellpadding="3" cellspacing="0" class="TABLE_G" id="ctl05_gv" rules="all" style="border-color:Black;border-width:1px;border-style:None;border-collapse:collapse;">
<tbody><tr style="color:Black;font-weight:normal;">
<th scope="col">監測項目</th><th scope="col">單位</th><th scope="col">監測日期</th><th scope="col">監測值</th><th scope="col">標註</th>
</tr><tr style="color:Black;">
<td class="no-alt" rowspan="12" style="white-space:nowrap;" valign="top">SO2</td><td class="no-alt" rowspan="12" valign="top">ppb</td><td>2018/01</td><td>1.80</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/02</td><td>1.90</td><td> </td>
</tr><tr style="color:Black;">
<td>2018/03</td><td>2.20</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/04</td><td>2.30</td><td> </td>
</tr><tr style="color:Black;">
<td>2018/05</td><td>3.10</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/06</td><td>2.70</td><td> </td>
</tr><tr style="color:Black;">
<td>

In [3]:
trs = table.findAll("tr")
dic = {}
now = ""
for tr in trs:
    tds = tr.findAll("td")
    if len(tds) == 5:
        now = tds[0].string
        dic[now]={}
        dic[now][tds[2].string]=tds[3].string
    elif len(tds) == 3:
        dic[now][tds[0].string]=tds[1].string
# print(dic["SO2"])
import pandas as pd
SO2_dic = [{'日期': key, '監測值': value} for key, value in dic['SO2'].items() if key <= '2018/08' ]
pd.DataFrame(SO2_dic)

,日期,監測值
0,2018/01,1.80
1,2018/02,1.90
2,2018/03,2.20
3,2018/04,2.30
4,2018/05,3.10
5,2018/06,2.70
6,2018/07,2.20
7,2018/08,2.40


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [4]:
SO2_dic = [{'日期': key, 'SO2': value} for key, value in dic['SO2'].items() if key <= '2018/08' ]
CO_dic = [{'日期': key, 'CO': value} for key, value in dic['CO'].items() if key <= '2018/08' ]
# print(SO2_dic)
pd.merge(pd.DataFrame(CO_dic),pd.DataFrame(SO2_dic))

,日期,CO,SO2
0,2018/01,0.34,1.80
1,2018/02,0.44,1.90
2,2018/03,0.40,2.20
3,2018/04,0.38,2.30
4,2018/05,0.34,3.10
5,2018/06,0.29,2.70
6,2018/07,0.21,2.20
7,2018/08,0.30,2.40
